# **Import Library**

In [177]:
import io
import re 
import unicodedata
import numpy as np
import pandas as pd
import matplotlib as plt
from typing import Optional

# **Load Dataset**

In [178]:
file_path = "datasets/data_inovasi.xlsx"
df = pd.read_excel(file_path)
df.head()

,No,Judul Inovasi,Pemda,Admin OPD,Inisiator,Nama Inisiator,Bentuk Inovasi,Jenis,Asta Cipta,Urusan Utama,Urusan lain yang beririsan,Kematangan,Tahapan Inovasi,Tanggal Input,Tanggal Penerapan,Tanggal Pengembangan,Koordinat,Video,Link Video
0,1,POP SURGA (Penghantaran Obat Pasien Sumberglagah),Provinsi Jawa Timur,Dinas Kesehatan UPT. RSK Sumberglagah Mojokert...,Kepala Daerah,"drg. SHINTA SAWITRI, M.Kes",Inovasi pelayanan publik,Digital,Memperkuat pembangunan sumber daya manusia (SD...,kesehatan,pekerjaan umum dan penataan ruang,85.0,Penerapan,22-07-2022,29-11-2023,-,"-7.60739065953564, 112.54078722662341",Ada,https://www.youtube.com/watch?v=o_TedznOu3U
1,2,PHEC (Pre Hospital Emergency Care),Provinsi Jawa Timur,Dinas Kesehatan UPT. RSK Sumberglagah Mojokert...,OPD,"drg. SHINTA SAWITRI, M.Kes",Inovasi pelayanan publik,Non Digital,Memperkuat pembangunan sumber daya manusia (SD...,pendidikan,kesehatan,92.0,Penerapan,22-07-2022,24-06-2023,-,"-7.6073109183015, 112.54072288029913",Ada,https://youtu.be/TJaII4_0UkI
2,3,Naskah Dinas Elektronik (NADINE),Provinsi Jawa Timur,Badan Koordinasi Wilayah Pamekasan Provinsi Ja...,OPD,"Dra. SUFI AGUSTINI, M.Si",Inovasi tata kelola pemerintahan daerah,Teknologi,"Memperkuat reformasi politik, hukum, dan birok...",kearsipan,Fungsi Penunjang lainnya sesuai dengan ketentu...,110.0,Penerapan,20-06-2023,31-12-2024,31-12-2024,"-7.158553824257441, 113.48272568540575",Ada,https://drive.google.com/file/d/192dhJGWtC4lDc...
3,4,PERMATA ( Pertanian Ramah Lingkungan menuju Ma...,Provinsi Jawa Timur,Dinas Pertanian dan Ketahanan Pangan (jatimpro...,OPD,-,Inovasi Daerah lainnya sesuai dengan Urusan Pe...,Digital,-,pertanian,NaN,60.0,Uji Coba,26-06-2023,25-09-2023,-,NaN,Tidak Ada,-
4,5,SIGALON,Provinsi Jawa Timur,Dinas Kesehatan UPT. RS Mata Masyarakat (jatim...,OPD,-,Inovasi pelayanan publik,Digital,-,kesehatan,NaN,52.0,Penerapan,10-07-2023,10-07-2023,-,NaN,Ada,https://drive.google.com/file/d/1JKCIqN4TRVL-O...


In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   No                          679 non-null    int64  
 1   Judul Inovasi               679 non-null    object 
 2   Pemda                       679 non-null    object 
 3   Admin OPD                   679 non-null    object 
 4   Inisiator                   679 non-null    object 
 5   Nama Inisiator              679 non-null    object 
 6   Bentuk Inovasi              679 non-null    object 
 7   Jenis                       679 non-null    object 
 8   Asta Cipta                  679 non-null    object 
 9   Urusan Utama                679 non-null    object 
 10  Urusan lain yang beririsan  463 non-null    object 
 11  Kematangan                  679 non-null    float64
 12  Tahapan Inovasi             679 non-null    object 
 13  Tanggal Input               679 non

# **Exploratory Data Analysis (EDA)**

In [180]:
# columns that have null values
null_columns = df.isnull().sum()
null_columns[null_columns > 0]

Urusan lain yang beririsan    216
Koordinat                      92
dtype: int64

In [181]:
# Cek nilai "-"
dash_count = (df == "-").sum()
dash_count[dash_count > 0].sort_values(ascending=False)

Tanggal Pengembangan    294
Link Video              108
Asta Cipta               92
Nama Inisiator           10
dtype: int64

In [182]:
# check uniqe value
unique_values = {col: df[col].nunique() for col in df.columns}
unique_values

{'No': 679,
 'Judul Inovasi': 664,
 'Pemda': 1,
 'Admin OPD': 292,
 'Inisiator': 4,
 'Nama Inisiator': 552,
 'Bentuk Inovasi': 3,
 'Jenis': 3,
 'Asta Cipta': 9,
 'Urusan Utama': 31,
 'Urusan lain yang beririsan': 132,
 'Kematangan': 89,
 'Tahapan Inovasi': 3,
 'Tanggal Input': 80,
 'Tanggal Penerapan': 378,
 'Tanggal Pengembangan': 243,
 'Koordinat': 418,
 'Video': 2,
 'Link Video': 569}

In [183]:
# Unique values for key categorical columns
categorical_cols = [
    "Bentuk Inovasi",
    "Jenis",
    "Tahapan Inovasi",
    "Inisiator",
    "Asta Cipta",
    "Urusan Utama",
    "Urusan lain yang beririsan"
]

for col in categorical_cols:
    print(f"\nDistribusi {col}:")
    temp = df[col].value_counts().reset_index()
    temp.columns = [col, "Jumlah"]
    print(temp)


Distribusi Bentuk Inovasi:
                                      Bentuk Inovasi  Jumlah
0                           Inovasi pelayanan publik     444
1  Inovasi Daerah lainnya sesuai dengan Urusan Pe...     185
2            Inovasi tata kelola pemerintahan daerah      50

Distribusi Jenis:
         Jenis  Jumlah
0  Non Digital     346
1      Digital     234
2    Teknologi      99

Distribusi Tahapan Inovasi:
  Tahapan Inovasi  Jumlah
0       Penerapan     648
1       Inisiatif      22
2        Uji Coba       9

Distribusi Inisiator:
       Inisiator  Jumlah
0            ASN     362
1            OPD     226
2  Kepala Daerah      49
3     Masyarakat      42

Distribusi Asta Cipta:
                                          Asta Cipta  Jumlah
0  Memperkuat pembangunan sumber daya manusia (SD...     400
1                                                  -      92
2  Meningkatkan lapangan kerja yang berkualitas, ...      62
3  Memperkuat penyelarasan kehidupan yang harmoni...      39
4  Memp

In [184]:
# df["Tahun"] = pd.to_datetime(df["Tanggal Penerapan"], errors="coerce").dt.year
# distribusi = (
#     df
#     .groupby(["Tahun", "Jenis"])
#     .size()
#     .reset_index(name="jumlah")
# )

# print(distribusi)

In [186]:
print("Jumlah data duplikat:", df.duplicated(keep=False).sum())

Jumlah data duplikat: 0


In [ ]:
# Check data Inovasi terduplikat
df_check = df.copy()
df_check["judul_raw"] = df_check["Judul Inovasi"].astype(str)

def text_normalize(text):
    text = str(text)
    # 1. Unicode normalize (hilangkan karakter aneh tersembunyi)
    text = unicodedata.normalize("NFKD", text)
    # 2. Lowercase
    text = text.lower()
    # 3. Samakan pemisah jadi spasi
    text = re.sub(r"[-_/]", " ", text)
    # 4. Hapus semua simbol & tanda baca → ganti spasi (BIAR KATA GA NEMPEL)
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    # 5. Rapikan spasi berlebih
    text = re.sub(r"\s+", " ", text).strip()
    # 6. Hapus kata umum (stopwords) — pakai word boundary biar aman
    stopwords = ["aplikasi", "sistem", "program", "inovasi", "digital", "berbasis"]
    pattern = r"\b(" + "|".join(stopwords) + r")\b"
    text = re.sub(pattern, " ", text)

    text = re.sub(r"\s+", " ", text).strip()
    
    return text

df_check["judul_super_norm"] = df_check["judul_raw"].apply(text_normalize)
dup_group = (
    df_check
    .groupby("judul_super_norm")
    .size()
    .reset_index(name="jumlah")
    .query("jumlah > 1")
)

print("Jumlah judul yang terduplikat:", len(dup_group))
dup_norm = df_check[df_check.duplicated(subset=["judul_super_norm"], keep=False)]

columns = [
    "judul_raw",
    "judul_super_norm",
    "Admin OPD",
    "Inisiator",
    "Nama Inisiator",
    "Jenis",
    "Kematangan",
    "Tahapan Inovasi",
    "Tanggal Input",
    "Tanggal Penerapan",
    "Tanggal Pengembangan"
]

df_duplicate = (
    dup_norm[columns]
    .sort_values(by=["judul_super_norm", "Admin OPD"])
    .reset_index(drop=True)
)

df_duplicate

Jumlah judul yang terduplikat: 25


,judul_raw,judul_super_norm,Admin OPD,Inisiator,Nama Inisiator,Jenis,Kematangan,Tahapan Inovasi,Tanggal Input,Tanggal Penerapan,Tanggal Pengembangan
0,BATA MERDEKA,bata merdeka,SMA Negeri 1 Jombang (iga2024.sma.n.1.jombang),ASN,"ZAGLUL RIJAL PASHA, S.Pd DKK",Non Digital,0.0,Penerapan,30-07-2025,01-11-2024,-
1,BATA MERDEKA,bata merdeka,SMA Negeri 1 Jombang (iga2024.sma.n.1.jombang),OPD,"ZAGHLUL RIJAL PHASA, S. Pd. DKK",Non Digital,101.0,Penerapan,01-08-2025,06-08-2024,-
2,"""BES-Q""(BERSIH ENTREPRENEUR SEHAT DAN QOLBU)",bes q bersih entrepreneur sehat dan qolbu,"SMAN 1 JENANGAN, PONOROGO (jatimprov.dispend...",OPD,"Titik Ruwaidah, S.Pd.,M.Pd.",Non Digital,96.0,Penerapan,26-07-2024,09-02-2024,26-04-2024
3,"""BES-Q"" (BERSIH ENTREPRENEUR SEHAT DAN QOLBU)",bes q bersih entrepreneur sehat dan qolbu,SMAN 1 JENANGAN (iga2024.sman.1.jenangan),OPD,"Titik Ruwaidah, S.Pd., M.Pd.",Non Digital,92.0,Penerapan,31-07-2024,03-03-2023,21-07-2023
4,DAKSA BUDAYA,daksa budaya,Dinas Kebudayaan dan Pariwisata Provinsi Jawa ...,OPD,"Evy Afianasari, S.T., M.M.A",Digital,111.0,Penerapan,20-08-2025,04-03-2024,04-03-2024
5,DAKSA BUDAYA,daksa budaya,admin.jawa.timur (iga2025.provinsi.jawa.timur),OPD,"Evy Afianasari, S.T, MMA",Digital,9.0,Penerapan,19-08-2025,02-01-2023,02-01-2024
6,DIGDAYA: Digitalisasi untuk Mewujudkan Kedisip...,digdaya digitalisasi untuk mewujudkan kedisipl...,SMAN 2 PONOROGO (jatimprov.dispendik.sman2pon...,ASN,"Zanwar Sugiartoko, S.Kom",Digital,107.0,Penerapan,29-07-2025,15-07-2024,-
7,DIGDAYA: Digitalisasi untuk Mewujudkan Kedisip...,digdaya digitalisasi untuk mewujudkan kedisipl...,SMAN 2 PONOROGO (iga2024.sman.2.ponorogo),ASN,"Zanwar Sugiartoko, S.Kom",Digital,99.0,Penerapan,05-08-2024,15-07-2024,-
8,‘E-KOMITE’ SISTEM MONITORING PELAYANAN PARTISI...,e komite monitoring pelayanan partisipasi sumb...,"SMK NEGERI 1 KARE, KAB MADIUN (jatimprov.dispe...",ASN,-,Digital,0.0,Penerapan,15-07-2023,27-09-2023,-
9,‘E-KOMITE’ SISTEM MONITORING PELAYANAN PARTISI...,e komite monitoring pelayanan partisipasi sumb...,"SMK NEGERI 1 KARE, KAB MADIUN (jatimprov.dispe...",ASN,"SEPTA KRISDIYANTO, M.Pd.",Digital,99.0,Penerapan,25-07-2024,27-09-2023,18-12-2023


# **Preprocessing**

In [188]:
df_clean = df.copy()

In [189]:
# merapikan nama kolom (lower case)
df_clean.columns = (
    df_clean.columns
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('.', '', regex=False)
)

string_cols = df_clean.select_dtypes(include="object").columns
for col in string_cols:
    df_clean[col] = df_clean[col].str.strip()

In [190]:
# Standarisasi missing value ("-" diganti NaN (kecuali link_video))
for col in df_clean.columns:
    if col != "link_video":
        df_clean[col] = df_clean[col].replace("-", np.nan)

In [191]:
# Konversi column date
date_cols = [
    "tanggal_input",
    "tanggal_pengembangan",
    "tanggal_penerapan"
]

for col in date_cols:
    df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")

C:\Users\Dicky Zulfikar\AppData\Local\Temp\ipykernel_9160\1711946224.py:9: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")
C:\Users\Dicky Zulfikar\AppData\Local\Temp\ipykernel_9160\1711946224.py:9: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")
C:\Users\Dicky Zulfikar\AppData\Local\Temp\ipykernel_9160\1711946224.py:9: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")


In [192]:
# Penanganan kolom kategorikal null
df_clean["urusan_lain_yang_beririsan"] = (
    df_clean["urusan_lain_yang_beririsan"]
    .fillna("Tidak Ada")
)

df_clean["nama_inisiator"] = (
    df_clean["nama_inisiator"]
    .replace("-", pd.NA)
    .fillna("Tidak Disebutkan")
)

df_clean["asta_cipta"] = (
    df_clean["asta_cipta"]
    .replace("-", pd.NA)
    .fillna("Tidak Diisi")
)

In [193]:
# Konversi numerik column 'kematangan'
df_clean['kematangan'] = pd.to_numeric(
    df_clean['kematangan'],
    errors='coerce'
)

# hanya hapus nilai negatif
df_clean.loc[
    df_clean['kematangan'] < 0,
    'kematangan'
] = np.nan

In [194]:
# hapus inovasi duplikat berdasarkan nilai kematangan tertinggi
def inovasi_normalize(text):
    text = str(text)
    text = unicodedata.normalize("NFKD", text)
    text = text.lower()
    text = re.sub(r"[-_/]", " ", text)          
    text = re.sub(r"[^a-z0-9\s]", " ", text)  
    text = re.sub(r"\s+", " ", text).strip()  

    stopwords = ["aplikasi", "sistem", "program", "inovasi", "digital", "berbasis"]
    pattern = r"\b(" + "|".join(stopwords) + r")\b"
    text = re.sub(pattern, " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

df_clean["judul_key"] = df_clean["judul_inovasi"].apply(inovasi_normalize)
idx = df_clean.groupby("judul_key")["kematangan"].idxmax()

df_clean = (
    df_clean.loc[idx] 
    .sort_index()      
    .drop(columns=["judul_key"])  
)

In [196]:
# labeling nilai kematangan
def label_kematangan(nilai):
    if pd.isna(nilai):
        return np.nan
    elif 0 <= nilai <= 44.99:
        return "Kurang Inovatif"
    elif 45 <= nilai <= 64.99:
        return "Inovatif"
    elif 65 <= nilai <= 111:
        return "Sangat Inovatif"
    else:
        return np.nan

df_clean["label_kematangan"] = df_clean["kematangan"].apply(label_kematangan)

In [197]:
# normalisasi koordinat (lat & lon)
def clean_coordinates(df: pd.DataFrame) -> pd.DataFrame:
    df['lat'] = np.nan
    df['lon'] = np.nan
    
    if 'koordinat' not in df.columns:
        return df

    def parse_coord(text):
        if pd.isna(text) or text == '' or text == '-':
            return np.nan, np.nan
            
        text = str(text).strip()
        match_decimal = re.search(r'(-?\d+\.\d+)[,\s]+.*?(-?\d+\.\d+)', text)
        if match_decimal:
            return float(match_decimal.group(1)), float(match_decimal.group(2))

        match_comma = re.findall(r'(-?\d+)', text)
        if len(match_comma) >= 4:
            try:
                lat = float(f"{match_comma[0]}.{match_comma[1]}")
                lon = float(f"{match_comma[2]}.{match_comma[3]}")
                if -11 <= lat <= -6 and 110 <= lon <= 116: 
                    return lat, lon
            except:
                pass
        return np.nan, np.nan

    coords = df['koordinat'].apply(parse_coord)

    df['lat'] = [c[0] for c in coords]
    df['lon'] = [c[1] for c in coords]
    df.drop(columns=['koordinat'], inplace=True)
    
    return df

df_clean = clean_coordinates(df_clean)
print(df_clean[['lat', 'lon']].info())
print("\nContoh data yang berhasil diekstrak:")
print(df_clean[['lat', 'lon']].dropna().head())

<class 'pandas.core.frame.DataFrame'>
Index: 653 entries, 0 to 678
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     517 non-null    float64
 1   lon     517 non-null    float64
dtypes: float64(2)
memory usage: 15.3 KB
None

Contoh data yang berhasil diekstrak:
         lat         lon
0  -7.607391  112.540787
1  -7.607311  112.540723
2  -7.158554  113.482726
5  -7.325387  112.731079
14 -6.993797  110.420183


In [198]:
# title case
title_case_cols = [
    "bentuk_inovasi",
    "urusan_utama",
    "urusan_lain_yang_beririsan",
]

for col in title_case_cols:
    df_clean[col] = (
        df_clean[col]
        .astype(str)
        .str.strip()
        .replace("nan", pd.NA)   
        .str.title()
    )

## Pipeline Preprocessing

In [199]:
def preprocessing_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    df_clean = df.copy()

    # 1. Rapikan nama kolom
    df_clean.columns = (
        df_clean.columns
        .str.lower()
        .str.replace(' ', '_')
        .str.replace('.', '', regex=False)
    )

    # 2. Strip string
    string_cols = df_clean.select_dtypes(include="object").columns
    for col in string_cols:
        df_clean[col] = df_clean[col].str.strip()

    # 3. Standarisasi missing value
    for col in df_clean.columns:
        if col != "link_video":
            df_clean[col] = df_clean[col].replace("-", np.nan)

    # 4. Konversi tanggal
    date_cols = ["tanggal_input", "tanggal_pengembangan", "tanggal_penerapan"]
    for col in date_cols:
        df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")

    # 5. Penanganan kategorikal
    df_clean["urusan_lain_yang_beririsan"] = df_clean["urusan_lain_yang_beririsan"].fillna("Tidak Ada")
    df_clean["nama_inisiator"] = df_clean["nama_inisiator"].fillna("Tidak Disebutkan")
    df_clean["asta_cipta"] = df_clean["asta_cipta"].fillna("Tidak Diisi")

    # 6. Kematangan numerik
    df_clean['kematangan'] = pd.to_numeric(df_clean['kematangan'], errors='coerce')
    df_clean.loc[df_clean['kematangan'] < 0, 'kematangan'] = np.nan

    # 🔥 7. Deduplikasi Judul Inovasi
    df_clean["judul_key"] = df_clean["judul_inovasi"].apply(inovasi_normalize)

    # Ambil baris dengan kematangan tertinggi per judul
    idx = df_clean.groupby("judul_key")["kematangan"].idxmax()

    df_clean = (
        df_clean.loc[idx]
        .sort_index()
        .drop(columns=["judul_key"])
    )

    # 8. Label kematangan
    def label_kematangan(nilai):
        if pd.isna(nilai):
            return np.nan
        elif 0 <= nilai <= 44.99:
            return "Kurang Inovatif"
        elif 45 <= nilai <= 64.99:
            return "Inovatif"
        elif 65 <= nilai <= 111:
            return "Sangat Inovatif"
        else:
            return np.nan

    df_clean["label_kematangan"] = df_clean["kematangan"].apply(label_kematangan)

    # 9. Koordinat
    df_clean = clean_coordinates(df_clean)

    # 10. Title case
    title_case_cols = ["bentuk_inovasi", "urusan_utama", "urusan_lain_yang_beririsan"]
    for col in title_case_cols:
        df_clean[col] = df_clean[col].astype(str).replace("nan", pd.NA).str.title()

    return df_clean

In [200]:
df_clean.head()

,no,judul_inovasi,pemda,admin_opd,inisiator,nama_inisiator,bentuk_inovasi,jenis,asta_cipta,urusan_utama,...,kematangan,tahapan_inovasi,tanggal_input,tanggal_penerapan,tanggal_pengembangan,video,link_video,label_kematangan,lat,lon
0,1,POP SURGA (Penghantaran Obat Pasien Sumberglagah),Provinsi Jawa Timur,Dinas Kesehatan UPT. RSK Sumberglagah Mojokert...,Kepala Daerah,"drg. SHINTA SAWITRI, M.Kes",Inovasi Pelayanan Publik,Digital,Memperkuat pembangunan sumber daya manusia (SD...,Kesehatan,...,85.0,Penerapan,2022-07-22,2023-11-29,NaT,Ada,https://www.youtube.com/watch?v=o_TedznOu3U,Sangat Inovatif,-7.607391,112.540787
1,2,PHEC (Pre Hospital Emergency Care),Provinsi Jawa Timur,Dinas Kesehatan UPT. RSK Sumberglagah Mojokert...,OPD,"drg. SHINTA SAWITRI, M.Kes",Inovasi Pelayanan Publik,Non Digital,Memperkuat pembangunan sumber daya manusia (SD...,Pendidikan,...,92.0,Penerapan,2022-07-22,2023-06-24,NaT,Ada,https://youtu.be/TJaII4_0UkI,Sangat Inovatif,-7.607311,112.540723
2,3,Naskah Dinas Elektronik (NADINE),Provinsi Jawa Timur,Badan Koordinasi Wilayah Pamekasan Provinsi Ja...,OPD,"Dra. SUFI AGUSTINI, M.Si",Inovasi Tata Kelola Pemerintahan Daerah,Teknologi,"Memperkuat reformasi politik, hukum, dan birok...",Kearsipan,...,110.0,Penerapan,2023-06-20,2024-12-31,2024-12-31,Ada,https://drive.google.com/file/d/192dhJGWtC4lDc...,Sangat Inovatif,-7.158554,113.482726
3,4,PERMATA ( Pertanian Ramah Lingkungan menuju Ma...,Provinsi Jawa Timur,Dinas Pertanian dan Ketahanan Pangan (jatimpro...,OPD,Tidak Disebutkan,Inovasi Daerah Lainnya Sesuai Dengan Urusan Pe...,Digital,Tidak Diisi,Pertanian,...,60.0,Uji Coba,2023-06-26,2023-09-25,NaT,Tidak Ada,-,Inovatif,NaN,NaN
4,5,SIGALON,Provinsi Jawa Timur,Dinas Kesehatan UPT. RS Mata Masyarakat (jatim...,OPD,Tidak Disebutkan,Inovasi Pelayanan Publik,Digital,Tidak Diisi,Kesehatan,...,52.0,Penerapan,2023-07-10,2023-07-10,NaT,Ada,https://drive.google.com/file/d/1JKCIqN4TRVL-O...,Inovatif,NaN,NaN


In [201]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 653 entries, 0 to 678
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   no                          653 non-null    int64         
 1   judul_inovasi               653 non-null    object        
 2   pemda                       653 non-null    object        
 3   admin_opd                   653 non-null    object        
 4   inisiator                   653 non-null    object        
 5   nama_inisiator              653 non-null    object        
 6   bentuk_inovasi              653 non-null    object        
 7   jenis                       653 non-null    object        
 8   asta_cipta                  653 non-null    object        
 9   urusan_utama                653 non-null    object        
 10  urusan_lain_yang_beririsan  653 non-null    object        
 11  kematangan                  653 non-null    float64       
 12 

In [202]:
df_clean.to_csv("datasets/data_inovasi_clean.csv", index=False)